<h1 align=center><font size = 5>Segmenting and Clustering Neighborhoods in Toronto</font></h1> 

<h3 align=left><font size = 5>1. Import libraries</font></h3> 

In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
# from bs4 import BeautifulSoup
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

import requests # Requests allows you to send organic, grass-fed HTTP/1.1 requests, without the need for manual labor.

print('Libraries imported.')

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.11
  latest version: 4.8.3

Please update conda by running

    $ conda update -n base -c defaults conda



## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs: 
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    pandas-0.24.2              |   py36hf484d3e_0        11.1 MB  conda-forge
    requests-2.23.0            |     pyh8c360ce_2          47 KB  conda-forge
    jsonschema-3.2.0           |   py36h9f0ad1d_1          89 KB  conda-forge
    altair-4.0.1               |             py_0         575 KB  conda-forge
    chardet-3.0.4              |py36h9f0ad1d_1006         188 KB  conda-forge
    folium-0.5.0               |             py_0          45 KB  conda-forge
    toolz-0.10.0               |             py_0   

In [2]:
!conda install lxml --yes

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.11
  latest version: 4.8.3

Please update conda by running

    $ conda update -n base -c defaults conda



## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs: 
    - lxml


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    lxml-4.3.0                 |   py36hefd8a0e_0         1.5 MB
    certifi-2019.11.28         |           py36_1         157 KB
    openssl-1.1.1e             |       h7b6447c_0         3.8 MB
    ------------------------------------------------------------
                                           Total:         5.4 MB

The following packages will be UPDATED:

    certifi: 2019.11.28-py36h9f0ad1d_1 conda-forge --> 2019.11.28-py36_1   
    lxml:    4.2.5-py37hefd8a0e_0                  --> 4.3.0-py36hefd8a0e_0
    opens

In [4]:
!conda update -c conda-forge -y pandas

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.11
  latest version: 4.8.3

Please update conda by running

    $ conda update -n base -c defaults conda



## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs: 
    - pandas


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    liblapack-3.8.0            |           14_mkl          10 KB  conda-forge
    libblas-3.8.0              |           14_mkl          10 KB  conda-forge
    pandas-1.0.3               |   py36h830a2c2_0        11.1 MB  conda-forge
    libcblas-3.8.0             |           14_mkl          10 KB  conda-forge
    numpy-1.18.1               |   py36h7314795_1         5.2 MB  conda-forge
    ------------------------------------------------------------
                                           Total:        16.4 MB

The f

In [6]:
conda install -c anaconda beautifulsoup4

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.11
  latest version: 4.8.3

Please update conda by running

    $ conda update -n base -c defaults conda



## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs: 
    - beautifulsoup4


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    soupsieve-2.0              |             py_0          33 KB  anaconda
    openssl-1.1.1              |       h7b6447c_0         5.0 MB  anaconda
    certifi-2019.11.28         |           py36_1         157 KB  anaconda
    beautifulsoup4-4.8.2       |           py36_0         161 KB  anaconda
    ------------------------------------------------------------
                                           Total:         5.4 MB

The following NEW packages will be INSTALLED:

    soupsieve:      2.0-py_0            

In [7]:
from bs4 import BeautifulSoup

In [2]:
import pandas as pd

<h2 align=left><font size = 5>2. Scraping table from Wikipedia page </font></h2> 
<h3 align=left><font size = 3>Use the BeautifulSoup package or <b>any other way</b> you are comfortable with </font></h3> 
<h3 align=left><font size = 3>I am using pd.read_html </font></h3> 



In [25]:
df = pd.read_html(r'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M',match= 'Neighborhood')
df = df[0]
df = pd.DataFrame(df) 
df.head()

,Postal code,Borough,Neighborhood
0,M1A,Not assigned,NaN
1,M2A,Not assigned,NaN
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Regent Park / Harbourfront


<h3 align=left><font size = 3> - The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood <br>
- Ignore cells with a borough that is Not assigned. </font></h3> 

In [26]:
df.columns= ["PostalCode","Borough","Neighborhood"] # The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood

In [27]:
df.drop(df.index[df['Borough'] == 'Not assigned'], inplace = True) # Ignore cells with a borough that is Not assigned.

In [29]:
df.head()

,PostalCode,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Regent Park / Harbourfront
5,M6A,North York,Lawrence Manor / Lawrence Heights
6,M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government


<h3 align=left><font size = 3> Merge cells with same postal code. <br>
More than one neighborhood can exist in one postal code area, separated them with ","  </font></h3> 

In [33]:
df=df.groupby(['PostalCode','Borough'],as_index=False).agg(lambda x: ','.join(x)) #Merge cells with same postal code.
#More than one neighborhood can exist in one postal code area, separated them with ","
df["Neighborhood"]= df["Neighborhood"].str.replace(" / ", ", ")
df.head()

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


<h3 align=left><font size = 3> If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough.  </font></h3> 

In [34]:
#If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough. 
e = len(df)
i = 1
while i < e:
    y = df.iat[i,2]
    
    if y == 'Not assigned':
        df.iat[i,2] = df.iat[i,1]
    i += 1
df.head()
# See Queen's Park (last row) for example

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


<h3 align=left><font size = 3> Show final result - with 15 random rows</font></h3> 

In [35]:
df_random = df.sample(n=15)
df_random

,PostalCode,Borough,Neighborhood
9,M1N,Scarborough,"Birch Cliff, Cliffside West"
46,M4R,Central Toronto,North Toronto West
89,M8W,Etobicoke,"Alderwood, Long Branch"
83,M6R,West Toronto,"Parkdale, Roncesvalles"
27,M3C,North York,Don Mills
53,M5A,Downtown Toronto,"Regent Park, Harbourfront"
38,M4G,East York,Leaside
51,M4X,Downtown Toronto,"St. James Town, Cabbagetown"
82,M6P,West Toronto,"High Park, The Junction South"
47,M4S,Central Toronto,Davisville


<h3 align=left><font size = 3> Use the .shape method to print the number of rows of your dataframe.  <br>
Total rows: 103</font></h3> 

In [36]:
df.shape[0]

103

<h3 align=left><font size = 3> Save Dataframe to CSV file</font></h3> 


In [ ]:
df.to_csv(r'Toronto.csv', index = False)

In [ ]:
coordinate = pd.read_csv('Geospatial_Coordinates.csv') # downloaded file from https://cocl.us/Geospatial_data
df=df.merge(coordinate,how='left',left_on='PostalCode',right_on='Postal Code')
del df['Postal Code'] # get rid of additional column
df